In [ ]:
import random
import pandas as pd

# Parameters
num_students = 300
num_supervisors = 10

# Combine all topics into a single list for random sampling
all_topics = topics_bcs_bse_bit + topics_bdsa + topics_bcns

# Generate students with topic preferences
students = []
for i in range(1, num_students + 1):
    programme = random.choice(list(programmes.keys()))  # Randomly assign a programme
    preferences = random.sample(programmes.get(programme), k=random.randint(1, len(programmes.get(programme)))) # Random subset of topics
    students.append({"StudentID": f"S{i}", "Preferences": preferences, "Programme": programme})

# Generate supervisors with topic preferences and capacities
supervisors = []
for i in range(1, num_supervisors + 1):
    topic_preferences = random.sample(all_topics, k=random.randint(1, len(all_topics) // 2)) # Random subset of topics
    programme_preferences = random.sample(sorted(programmes.keys()), k=2)
    capacity = random.randint(2, 5)  # Supervisors can supervise 2-5 students
    supervisors.append({"SupervisorID": f"SP{i}", "Programme Preferences": programme_preferences, "Topic Preferences": topic_preferences, "Capacity": capacity})

# Convert to DataFrames for better visualization
students_df = pd.DataFrame(students)
supervisors_df = pd.DataFrame(supervisors)

# Display the datasets
print("Students Dataset:")
print(students_df.head())
print("\nSupervisors Dataset:")
print(supervisors_df.head())

Students Dataset:
  StudentID                                        Preferences Programme
0        S1  [Computer Networks, Data Structures, Machine L...       BCS
1        S2  [Cloud Computing, Cybersecurity, Computer Netw...       BSE
2        S3    [Deep Learning, Machine Learning, Data Science]      BDSA
3        S4  [Cybersecurity, Network Security, Ethical Hack...      BCNS
4        S5  [Algorithms, Data Structures, Machine Learning...       BCS

Supervisors Dataset:
  SupervisorID Programme Preferences  \
0          SP1           [BSE, BCNS]   
1          SP2           [BDSA, BCS]   
2          SP3           [BDSA, BIT]   
3          SP4           [BDSA, BSE]   
4          SP5           [BSE, BDSA]   

                                   Topic Preferences  Capacity  
0  [Software Engineering, Data Analytics, Cloud S...         5  
1  [Mobile App Development, Data Science, Data Vi...         3  
2  [Statistical Modeling, Data Analytics, Mobile ...         2  
3  [Software Engineer

In [2]:
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

# Create the optimization problem
problem = LpProblem("Optimal_Matching", LpMaximize)

# Create decision variables for each student-supervisor pair
decision_vars = {}
for student in students:
    for supervisor in supervisors:
        decision_vars[(student["StudentID"], supervisor["SupervisorID"])] = LpVariable(
            f"x_{student['StudentID']}_{supervisor['SupervisorID']}", 0, 1, LpBinary
        )

# Objective function: Maximize the total preference match score
problem += lpSum(
    decision_vars[(student["StudentID"], supervisor["SupervisorID"])]
    for student in students
    for supervisor in supervisors
    if (
        student["Programme"] in supervisor["Programme Preferences"] and
        any(topic in supervisor["Topic Preferences"] for topic in student["Preferences"])
    )
)

# Constraint: Each student is assigned to exactly one supervisor
for student in students:
    problem += lpSum(
        decision_vars[(student["StudentID"], supervisor["SupervisorID"])]
        for supervisor in supervisors
    ) == 1

# Constraint: Each supervisor does not exceed their capacity
for supervisor in supervisors:
    problem += lpSum(
        decision_vars[(student["StudentID"], supervisor["SupervisorID"])]
        for student in students
    ) <= supervisor["Capacity"]

# Solve the problem
problem.solve()

# Extract the results
assignments = []
for student in students:
    for supervisor in supervisors:
        if decision_vars[(student["StudentID"], supervisor["SupervisorID"])].value() == 1:
            assignments.append((student["StudentID"], supervisor["SupervisorID"]))

# Display the assignments
print("Optimal Assignments:")
for student_id, supervisor_id in assignments:
    print(f"Student {student_id} -> Supervisor {supervisor_id}")

Optimal Assignments:
Student S23 -> Supervisor SP2
Student S24 -> Supervisor SP10
Student S25 -> Supervisor SP2
Student S217 -> Supervisor SP2
Student S238 -> Supervisor SP5
Student S239 -> Supervisor SP2
Student S240 -> Supervisor SP1
Student S241 -> Supervisor SP1
Student S242 -> Supervisor SP1
Student S243 -> Supervisor SP7
Student S244 -> Supervisor SP1
Student S245 -> Supervisor SP8
Student S246 -> Supervisor SP10
Student S247 -> Supervisor SP10
Student S248 -> Supervisor SP9
Student S249 -> Supervisor SP4
Student S250 -> Supervisor SP10
Student S251 -> Supervisor SP4
Student S252 -> Supervisor SP7
Student S253 -> Supervisor SP7
Student S254 -> Supervisor SP6
Student S256 -> Supervisor SP7
Student S258 -> Supervisor SP4
Student S259 -> Supervisor SP6
Student S260 -> Supervisor SP3
Student S261 -> Supervisor SP5
Student S262 -> Supervisor SP8
Student S265 -> Supervisor SP1
